In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pd.set_option('display.max_columns', None)

# EDA

In [2]:
!ls ./data

Description.xlsx test_final.csv   train_final.csv


In [3]:
df_train = pd.read_csv('./data/train_final.csv')
df_test = pd.read_csv('./data/test_final.csv')

In [4]:
df_train.head()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,discrete_addr_state_20_one_hot,discrete_addr_state_21_one_hot,discrete_addr_state_22_one_hot,discrete_addr_state_23_one_hot,discrete_addr_state_24_one_hot,discrete_addr_state_25_one_hot,discrete_addr_state_26_one_hot,discrete_addr_state_27_one_hot,discrete_addr_state_28_one_hot,discrete_addr_state_29_one_hot,discrete_addr_state_30_one_hot,discrete_addr_state_31_one_hot,discrete_addr_state_32_one_hot,discrete_addr_state_33_one_hot,discrete_addr_state_34_one_hot,discrete_addr_state_35_one_hot,discrete_addr_state_36_one_hot,discrete_addr_state_37_one_hot,discrete_addr_state_38_one_hot,discrete_addr_state_39_one_hot,discrete_addr_state_40_one_hot,discrete_addr_state_41_one_hot,discrete_addr_state_42_one_hot,discrete_addr_state_43_one_hot,discrete_addr_state_44_one_hot,discrete_addr_state_45_one_hot,discrete_addr_state_46_one_hot,discrete_addr_state_47_one_hot,discrete_addr_state_48_one_hot,discrete_addr_state_49_one_hot,discrete_application_type_1_one_hot,discrete_application_type_2_one_hot,discrete_emp_length_1_one_hot,discrete_emp_length_2_one_hot,discrete_emp_length_3_one_hot,discrete_emp_length_4_one_hot,discrete_emp_length_5_one_hot,discrete_emp_length_6_one_hot,discrete_emp_length_7_one_hot,discrete_emp_length_8_one_hot,discrete_emp_length_9_one_hot,discrete_emp_length_10_one_hot,discrete_emp_length_11_one_hot,discrete_emp_length_12_one_hot,discrete_grade_1_one_hot,discrete_grade_2_one_hot,discrete_grade_3_one_hot,discrete_grade_4_one_hot,discrete_grade_5_one_hot,discrete_grade_6_one_hot,discrete_grade_7_one_hot,discrete_home_ownership_1_one_hot,discrete_home_ownership_2_one_hot,discrete_home_ownership_3_one_hot,discrete_home_ownership_4_one_hot,discrete_policy_code_1_one_hot,discrete_purpose_1_one_hot,discrete_purpose_2_one_hot,discrete_purpose_3_one_hot,discrete_purpose_4_one_hot,discrete_purpose_5_one_hot,discrete_purpose_6_one_hot,discrete_purpose_7_one_hot,discrete_purpose_8_one_hot,discrete_purpose_9_one_hot,discrete_purpose_10_one_hot,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,d

In [5]:
df_train.loan_status.value_counts()

1    39788
0    10212
Name: loan_status, dtype: int64

In [6]:
df_train.info(null_counts=True,verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 146 columns):
continuous_annual_inc                     50000 non-null float64
continuous_annual_inc_joint               220 non-null float64
continuous_delinq_2yrs                    50000 non-null float64
continuous_dti                            49999 non-null float64
continuous_dti_joint                      220 non-null float64
continuous_fico_range_high                50000 non-null float64
continuous_fico_range_low                 50000 non-null float64
continuous_funded_amnt                    50000 non-null float64
continuous_funded_amnt_inv                50000 non-null float64
continuous_inq_last_6mths                 50000 non-null float64
continuous_installment                    50000 non-null float64
continuous_int_rate                       50000 non-null float64
continuous_last_fico_range_high           50000 non-null float64
continuous_last_fico_range_low            50000 n

# baseline

In [101]:
#训练集 测试集准备
train_data = df_train.copy()
test_data = df_test.copy()
train_y = train_data.loan_status
test_y = test_data.loan_status
train_data.drop(['loan_status'],axis=1,inplace=True)
test_data.drop(['loan_status'], axis=1,inplace=True)
#祖传参数
params = {
    'boosting_type':"gbdt",'num_leaves':31, 'reg_alpha':0, 'reg_lambda':0.,
    'max_depth':-1, 'n_estimators':2500, 'objective':'binary',
    'subsample':0.9, 'colsample_bytree':0.5, 'subsample_freq':1,
    'learning_rate':0.035, 'random_state':'2021','metric': {'l2', 'auc'},
#     'is_unbalance':True

}

In [102]:
lgb_train = lgb.Dataset(train_data, train_y)
lgb_eval = lgb.Dataset(test_data, test_y, reference=lgb_train)

In [103]:
# lgb.cv(params, lgb_train, num_boost_round=10, nfold=5)

In [104]:
lgb_model = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=5)

/Library/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.150363	valid_0's auc: 0.94922
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.144217	valid_0's auc: 0.951929
[3]	valid_0's l2: 0.138343	valid_0's auc: 0.9522
[4]	valid_0's l2: 0.132807	valid_0's auc: 0.952854
[5]	valid_0's l2: 0.127654	valid_0's auc: 0.953017
[6]	valid_0's l2: 0.122849	valid_0's auc: 0.953276
[7]	valid_0's l2: 0.118414	valid_0's auc: 0.955122
[8]	valid_0's l2: 0.114312	valid_0's auc: 0.955147
[9]	valid_0's l2: 0.110511	valid_0's auc: 0.9552
[10]	valid_0's l2: 0.109721	valid_0's auc: 0.956471
[11]	valid_0's l2: 0.106203	valid_0's auc: 0.956526
[12]	valid_0's l2: 0.102974	valid_0's auc: 0.956576
[13]	valid_0's l2: 0.0999448	valid_0's auc: 0.956554
[14]	valid_0's l2: 0.0994432	valid_0's auc: 0.956277
[15]	valid_0's l2: 0.0966654	valid_0's auc: 0.956304
[16]	valid_0's l2: 0.0940995	valid_0's auc: 0.956528
[17]	valid_0's l2: 0.0917236	valid_0's auc: 0.956567
Early stopping, best iteration is:
[12]	valid_0's l2: 0.102974	va

In [105]:
test_y_pred_proba = lgb_model.predict(test_data)
test_y_pred = [1 if i >= 0.5 else 0 for i in test_y_pred_proba]
print('test metric of baseline')
print('test metric of baseline after opted\n')
print('Test accuracy is: ',round(accuracy_score(test_y, test_y_pred),3))
print('Test f1_score is: ',round(f1_score(test_y, test_y_pred),3))
print('Test precision is: ',round(precision_score(test_y, test_y_pred),3))
print('Test recall is: ',round(recall_score(test_y, test_y_pred),3))
# print(classification_report(test_y,test_y_pred))

test metric of baseline
test metric of baseline after opted

Test accuracy is:  0.805
Test f1_score is:  0.892
Test precision is:  0.805
Test recall is:  1.0


# 调参

In [8]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [70])
    metric: any = hp.choice('metric', ['binary_error'])
    n_estimators: any = hp.choice('n_estimators', [100])
    min_child_samples: any = hp.choice('min_child_samples', [20,30,40,50,60,70])
    max_depth: any = hp.choice('max_depth', [6])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.6, 0.8)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.6, 0.8)
    bagging_freq: any = hp.choice('bagging_freq', [1])
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt'])
    extra_trees: any = hp.choice('extra_tress', [False])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'boosting_type': 'gbdt', 'num_leaves': 70, 'metric': 'binary_error', 'objective': 'binary',
                'n_estimators': 100, 'max_depth': 6, 'bagging_freq': 1, 'min_child_samples': 20}

In [9]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")

In [10]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        n_estimators = use_params.pop('n_estimators')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, n_estimators, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, n_estimators, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = n_estimators
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [63]:
fitter = LGBFitter(label='loan_status')

In [13]:
from sklearn.model_selection import KFold
np.random.seed(2021)
kfold = KFold(n_splits=5)

In [68]:
fitter.search_k_fold(kfold, df_train)

In [75]:
from hyperopt import space_eval
space_eval(asdict(fitter.opt), fitter.opt_params)

{'bagging_fraction': 0.7858521175327225,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'device_type': 'cpu',
 'drop_rate': 0.13536062836550203,
 'extra_trees': False,
 'feature_fraction': 0.7102379179437247,
 'lambda_l1': 8.950471961978664,
 'lambda_l2': 5.919875366059117,
 'learning_rate': 0.09381607574376198,
 'max_depth': 6,
 'metric': 'binary_error',
 'min_child_samples': 30,
 'min_gain_to_split': 0.010671221839055611,
 'n_estimators': 100,
 'num_leaves': 70,
 'num_threads': 4,
 'objective': 'binary',
 'uniform_drop': False}

## 调参后baseline

In [84]:
opted_params = {'bagging_fraction': 0.7858521175327225,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'device_type': 'cpu',
 'drop_rate': 0.13536062836550203,
 'extra_trees': False,
 'feature_fraction': 0.7102379179437247,
 'lambda_l1': 8.950471961978664,
 'lambda_l2': 5.919875366059117,
 'learning_rate': 0.09381607574376198,
 'max_depth': 6,
 'metric': {'binary_error','l2', 'auc'},
 'min_child_samples': 30,
 'min_gain_to_split': 0.010671221839055611,
 'n_estimators': 100,
 'num_leaves': 70,
 'num_threads': 4,
 'objective': 'binary',
 'uniform_drop': False,
#  'is_unbalance':True
               }
lgb_model = lgb.train(opted_params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=5)

[1]	valid_0's l2: 0.13933	valid_0's binary_error: 0.19548	valid_0's auc: 0.946502
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.124989	valid_0's binary_error: 0.19548	valid_0's auc: 0.948006
[3]	valid_0's l2: 0.11337	valid_0's binary_error: 0.19548	valid_0's auc: 0.94898
[4]	valid_0's l2: 0.103974	valid_0's binary_error: 0.19548	valid_0's auc: 0.951857
[5]	valid_0's l2: 0.0962557	valid_0's binary_error: 0.19548	valid_0's auc: 0.95199
[6]	valid_0's l2: 0.0899539	valid_0's binary_error: 0.1401	valid_0's auc: 0.952225
[7]	valid_0's l2: 0.0847293	valid_0's binary_error: 0.10568	valid_0's auc: 0.952448
[8]	valid_0's l2: 0.0804913	valid_0's binary_error: 0.09566	valid_0's auc: 0.95257
[9]	valid_0's l2: 0.0769567	valid_0's binary_error: 0.09122	valid_0's auc: 0.952797
[10]	valid_0's l2: 0.0763265	valid_0's binary_error: 0.09156	valid_0's auc: 0.955101
[11]	valid_0's l2: 0.0734961	valid_0's binary_error: 0.08874	valid_0's auc: 0.955391
[12]	valid_0's l2: 0.071

In [98]:
test_y_pred_proba = lgb_model.predict(test_data)
test_y_pred = [1 if i >= 0.5 else 0 for i in test_y_pred_proba]
print('test metric of baseline after opted\n')
print('Test accuracy is: ',round(accuracy_score(test_y, test_y_pred),3))
print('Test f1_score is: ',round(f1_score(test_y, test_y_pred),3))
print('Test precision is: ',round(precision_score(test_y, test_y_pred),3))
print('Test recall is: ',round(recall_score(test_y, test_y_pred),3))
# print(classification_report(test_y,test_y_pred))

test metric of baseline after opted

Test accuracy is:  0.917
Test f1_score is:  0.948
Test precision is:  0.948
Test recall is:  0.949


# 增加衍生变量

In [5]:
continuous_features = [name for name in df_train.columns if 'continuous' in name]

In [14]:
def feature_derivation(data_df):
    # 离散化 分箱
    for f in continuous_features:
        data_df[f+'_5_bin'] = pd.cut(data_df[f], 5, duplicates='drop').apply(lambda x:x.left).astype(int)
        data_df[f+'_10_bin'] = pd.cut(data_df[f], 10, duplicates='drop').apply(lambda x:x.left).astype(int)
        data_df[f+'_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x:x.left).astype(int)
    return data_df
df_fd_train = feature_derivation(df_train)
df_fd_test = feature_derivation(df_test)


fd_train_data = df_fd_train.copy()
fd_test_data = df_fd_train.copy()
fd_train_y = fd_train_data.loan_status
fd_test_y = fd_train_data.loan_status
fd_train_data.drop(['loan_status'],axis=1,inplace=True)
fd_test_data.drop(['loan_status'], axis=1,inplace=True)

fd_lgb_train = lgb.Dataset(fd_train_data, fd_train_y,free_raw_data = False)
fd_lgb_eval = lgb.Dataset(fd_test_data, fd_test_y, reference=fd_lgb_train)

In [16]:
fd_fitter = LGBFitter(label='loan_status')
fd_fitter.search_k_fold(kfold, df_fd_train)

In [127]:
space_eval(asdict(fd_fitter.opt), fd_fitter.opt_params)

{'bagging_fraction': 0.7597096327498998,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'device_type': 'cpu',
 'drop_rate': 0.07738912939781958,
 'extra_trees': False,
 'feature_fraction': 0.6371608764182851,
 'lambda_l1': 7.470561343331815,
 'lambda_l2': 2.5056087530642985,
 'learning_rate': 0.0939360618844934,
 'max_depth': 6,
 'metric': 'binary_error',
 'min_child_samples': 20,
 'min_gain_to_split': 0.7533927857756708,
 'n_estimators': 100,
 'num_leaves': 70,
 'num_threads': 4,
 'objective': 'binary',
 'uniform_drop': False}

In [18]:
opted_fd_params = {'bagging_fraction': 0.7597096327498998,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'device_type': 'cpu',
 'drop_rate': 0.07738912939781958,
 'extra_trees': False,
 'feature_fraction': 0.6371608764182851,
 'lambda_l1': 7.470561343331815,
 'lambda_l2': 2.5056087530642985,
 'learning_rate': 0.0939360618844934,
 'max_depth': 6,
 'metric': {'binary_error','l2', 'auc'},
 'min_child_samples': 20,
 'min_gain_to_split': 0.7533927857756708,
 'n_estimators': 100,
 'num_leaves': 70,
 'num_threads': 4,
 'objective': 'binary',
 'uniform_drop': False
               }
fd_lgb_model = lgb.train(opted_fd_params, fd_lgb_train, num_boost_round=20, valid_sets=fd_lgb_eval, early_stopping_rounds=5)

/Library/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.950272	valid_0's binary_error: 0.20424	valid_0's l2: 0.143373
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.954719	valid_0's binary_error: 0.20424	valid_0's l2: 0.12824
[3]	valid_0's auc: 0.955368	valid_0's binary_error: 0.20424	valid_0's l2: 0.115985
[4]	valid_0's auc: 0.955698	valid_0's binary_error: 0.20424	valid_0's l2: 0.106074
[5]	valid_0's auc: 0.956147	valid_0's binary_error: 0.20424	valid_0's l2: 0.0979856
[6]	valid_0's auc: 0.956195	valid_0's binary_error: 0.14652	valid_0's l2: 0.0913241
[7]	valid_0's auc: 0.956475	valid_0's binary_error: 0.10864	valid_0's l2: 0.0858307
[8]	valid_0's auc: 0.956659	valid_0's binary_error: 0.09346	valid_0's l2: 0.0812902
[9]	valid_0's auc: 0.956835	valid_0's binary_error: 0.08922	valid_0's l2: 0.0774215
[10]	valid_0's auc: 0.958573	valid_0's binary_error: 0.08734	valid_0's l2: 0.0742346
[11]	valid_0's auc: 0.95871	valid_0's binary_error: 0.08538	valid_0's l2: 0.0715628
[12]	valid_0's auc: 

In [19]:
test_y_pred_proba = fd_lgb_model.predict(fd_test_data)
test_y_pred = [1 if i >= 0.5 else 0 for i in test_y_pred_proba]
print('test metric of baseline after opted\n')
print('Test accuracy is: ',round(accuracy_score(fd_test_y, test_y_pred),3))
print('Test f1_score is: ',round(f1_score(fd_test_y, test_y_pred),3))
print('Test precision is: ',round(precision_score(fd_test_y, test_y_pred),3))
print('Test recall is: ',round(recall_score(fd_test_y, test_y_pred),3))
# print(classification_report(test_y,test_y_pred))

test metric of baseline after opted

Test accuracy is:  0.922
Test f1_score is:  0.951
Test precision is:  0.951
Test recall is:  0.951


# conclusion

简单的数据分箱后的衍生数据使得baseline的acc从0.917到了0.922